### 分治法

divide：将问题分割成同性质小规模子问题

conquer：递归解决子问题

combine：将子问题结果拼接

大体结构为

In [ ]:
def func(array):
    n = len(array)
    res1 = func(array[:(n//2)])
    res2 = func(array[(n//2):])
    return combine(res1, res2)

### 时间复杂度计算

Master's Theorem：已知分治法的时间复杂度满足$T(n)=aT(n/b)+f(n)$，那么

1.如果存在$\epsilon >0$，使得$f(n)=O(n^{\log_ba-\epsilon})$，那么$T(n)=\Theta(n^{\log_ba})$

2.如果$f(n)=\Theta(n^{\log_ba})$，那么$T(n)=\Theta(n^{\log_ba\log n})$

3.如果存在$\epsilon >0$，使得$f(n)=\Omega(n^{\log_ba+\epsilon})$，且存在$c<1$，使得$af(n/b)\leq cf(n)$，那么$T(n)=\Theta(f(n))$

### 算法题1:最大子序列

已知一只股票在一段时间内每个时间点的价格，你可以选择一个时间点买入该股票，并选择之后的一个时间点卖出该股票，如何买能够最大化收益？

注：下标从0开始

例：股票价格为[10, 11, 7, 10, 6]，在时刻2（价格为7）买入，时刻3（价格为10）卖出，收益最高

**暴力解法：** 遍历所有可能的买入时刻和卖出时刻的组合，时间复杂度为$O(n^2)$

In [1]:
def buy_stock(array):
    n = len(array)
    res = float("-inf")
    for i in range(n):
        for j in range(i+1, n):
            if array[j] - array[i] > res:
                left = i
                right = j
                res = array[j] - array[i]
    return res, left, right

我们想办法改进该算法的时间复杂度，考虑一些贪心方法。首先，我们考虑取最高价格和最低价格。遗憾的是，最高价可能先于最低价出现。然后，我们猜想是不是最优解一定包含最高价和最低价中的至少一个，但是考虑[10, 11, 7, 10, 6]，最优解是7->10，但是7不是最低价，10也不是最高价。上述贪心法不适用于该题目，我们考虑用分治法解决该问题。

**分治法：** 我们首先将该问题进行转换，该问题的本质并不关注价格本身，而是关注价格的变化趋势。例如：[10, 11, 7, 10, 6]和[11, 12, 8, 11, 7]的解一定是一样的。所以我们对原数据做一个差分，即将[10, 11, 7, 10, 6]转化为[1, -4, 3, -4]。那么问题转化为：***给定一个序列，找到一个连续子序列，使得该子序列的和最大***。

假设有一个序列，最小下标为low，最大下标high，选取一个中间节点mid，最大子序列只有三种情况：

情况1: 位于[low, mid]中，时间复杂度为$T(n/2)$

情况2: 位于[mid+1, high]中，时间复杂度为$T(n/2)$
 
情况3: 包括mid和mid+1，时间复杂度为$\Theta(n)$

可以对三种情况分别求出最大子序列，比较哪个和最大用哪个。其中，情况1和情况2可以分治，情况3只需要分别从mid处向左和mid+1处向右取最大和，拼接得到。

In [2]:
def maximum_subarray(array, low, high):
    ## base情形
    if low == high:
        return array[low], low, high
    mid = (low + high) // 2
    
    s1, left1, right1 = maximum_subarray(array, low, mid)
    s2, left2, right2 = maximum_subarray(array, mid + 1, high)
    s3, left3, right3 = maximum_crossing_subarray(array, low, high, mid)
    
    if s1 >= s2 and s1 >= s3:
        return s1, left1, right1
    elif s2 >= s1 and s2 >= s3:
        return s2, left2, right2
    else:
        return s3, left3, right3
    
def maximum_crossing_subarray(array, low, high, mid):
    left_sum = float("-inf")
    s = 0
    for i in range(mid, low-1, -1):
        s += array[i]
        if s > left_sum:
            left = i
            left_sum = s
    
    right_sum = float("-inf")
    s = 0
    for i in range(mid, high+1):
        s += array[i]
        if s > right_sum:
            right = i
            right_sum = s
    
    return left_sum + right_sum, left, right

时间复杂度分析：根据三种情况的拼接，我们得到$T(n)=2T(n/2)+\Theta(n)$，根据master's theorem的第2条，我们得到$T(n)=\Theta(n\log n)$